In [1]:
from selenium import webdriver
import pandas as pd
import time
import random

##### Готовим первичный набор ссылок и заголовков
Парсинг заголовков и ссылок, находящихся на странице https://dommagii.com/horoscop/archive

In [41]:
driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')
driver.get("https://dommagii.com/horoscop/archive")

In [42]:
titles = []
links = []
elems = driver.find_elements_by_tag_name("a")

In [43]:
for el in elems:
    try:
        if "-" in el.get_attribute("href"):
            titles.append(el.text)
            links.append(el.get_attribute("href"))
    except:
        continue

In [44]:
for i, j in enumerate(zip(titles, links)):
    if i % 100 == 0:
        print(j)

('Актуально', 'https://dommagii.com/breaking-news')
('Гороскоп на 3 июня 2018 год', 'https://dommagii.com/horoscop/2018-06-03')
('Гороскоп на 15 января 2017 год', 'https://dommagii.com/horoscop/2017-01-15')
('Гороскоп на 5 мая 2017 год', 'https://dommagii.com/horoscop/2017-05-05')
('Гороскоп на 22 октября 2017 год', 'https://dommagii.com/horoscop/2017-10-22')
('Гороскоп на 2 апреля 2016 года', 'https://dommagii.com/horoscop/2016-04-02')
('Гороскоп на 16 января 2015 года', 'https://dommagii.com/horoscop/2015-01-16')
('Гороскоп на 25 апреля 2015 года', 'https://dommagii.com/horoscop/2015-04-25')
('Гороскоп на 3 августа 2015 года', 'https://dommagii.com/horoscop/2015-08-03')
('Гороскоп на 13 ноября 2015 года', 'https://dommagii.com/horoscop/2015-11-13')
('Гороскоп на 20 февраля 2014', 'https://dommagii.com/horoscop/2014-02-20')
('Гороскоп на 31 мая 2015 года', 'https://dommagii.com/horoscop/2014-05-31-0')
('Гороскоп на 7 сентября 2014', 'https://dommagii.com/horoscop/2014-09-07')
('Гороск

In [98]:
df_one = pd.DataFrame(data=[titles, links], index=['titles', 'links']).T
df_one[::500]

,titles,links
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06
500,Гороскоп на 19 июня 2016 года,https://dommagii.com/horoscop/2016-06-19
1000,Гороскоп на 13 марта 2014,https://dommagii.com/horoscop/2014-03-13
1500,Гороскоп на 31 июля 2013,https://dommagii.com/horoscop/2013-07-31
2000,Гороскоп на 26 декабря 2012,https://dommagii.com/horoscop/2012-12-26


In [97]:
df_one = df_one.loc[df_one['titles'].str.contains('Гороскоп на ')][2:-2]
df_one[::500]

,titles,links
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06
500,Гороскоп на 19 июня 2016 года,https://dommagii.com/horoscop/2016-06-19
1000,Гороскоп на 13 марта 2014,https://dommagii.com/horoscop/2014-03-13
1500,Гороскоп на 31 июля 2013,https://dommagii.com/horoscop/2013-07-31
2000,Гороскоп на 26 декабря 2012,https://dommagii.com/horoscop/2012-12-26


In [53]:
df_one.to_csv('df_one_dommagii.csv', index=False)

In [54]:
driver.close()

##### Теперь проходим по собранным ссылкам

In [57]:
driver = webdriver.Chrome('C:\chromedriver_win32\chromedriver.exe')

In [58]:
df_one = pd.read_csv('df_one_dommagii.csv')

In [75]:
import tqdm

In [84]:
df_three = df_one.copy()
df_three['articles'] = ''
df_three['date'] = df_three['links'].apply(lambda x: x[-10:])

for index, row in tqdm.tqdm_notebook(df_three.loc[df_three['articles']==''].iterrows()):
    try:
        driver.get(row['links'])
    #     time.sleep(random.random() * 5)
        block = driver.find_elements_by_class_name("horo_block_wrapper")
        content = [el.text for el in block]
        df_three.loc[index, 'articles'] = content
    except:
        continue

In [85]:
df_three.head()

,titles,links,articles,date
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06,[ОВЕН (21.03 - 20.04)\nВы наполняетесь энтузиа...,2018-01-06
1,Гороскоп на 7 января 2018 год,https://dommagii.com/horoscop/2018-01-07,[ОВЕН (21.03 - 20.04)\nПоскольку ваша правящая...,2018-01-07
2,Гороскоп на 17 января 2018 год,https://dommagii.com/horoscop/2018-01-17,"[ОВЕН (21.03 - 20.04)\nСложный, но в тоже врем...",2018-01-17
3,Гороскоп на 18 января 2018 год,https://dommagii.com/horoscop/2018-01-18,[ОВЕН (21.03 - 20.04)\nАстрологическое руковод...,2018-01-18
4,Гороскоп на 19 января 2018 год,https://dommagii.com/horoscop/2018-01-19,[ОВЕН (21.03 - 20.04)\nВаше конфликтное поведе...,2018-01-19


In [90]:
df_three.to_csv('df_three_dommagii_draft.csv', index=False, sep=';')

In [96]:
df_three = df_three.loc[df_three['articles']!='']
df_three[::500]

,titles,links,articles,date,len_articles
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06,[ОВЕН (21.03 - 20.04)\nВы наполняетесь энтузиа...,2018-01-06,12
500,Гороскоп на 19 июня 2016 года,https://dommagii.com/horoscop/2016-06-19,"[ОВЕН (21.03 - 20.04)\nВаш потенциал, особенно...",2016-06-19,12
1000,Гороскоп на 13 марта 2014,https://dommagii.com/horoscop/2014-03-13,[ОВЕН (21.03 - 20.04)\nОчень сложно сегодня уд...,2014-03-13,12
1500,Гороскоп на 31 июля 2013,https://dommagii.com/horoscop/2013-07-31,[ОВЕН (21.03 - 20.04)\nЗвезды решили проверить...,2013-07-31,12
2000,Гороскоп на 26 декабря 2012,https://dommagii.com/horoscop/2012-12-26,[ОВЕН (21.03 - 20.04)\nОвны в этот день получа...,2012-12-26,12


In [92]:
driver.close()

In [93]:
df_three.loc[df_three['articles']=='']

,titles,links,articles,date


In [94]:
df_three['len_articles'] = df_three['articles'].apply(lambda x: len(x))

In [95]:
df_three[::500]

,titles,links,articles,date,len_articles
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06,[ОВЕН (21.03 - 20.04)\nВы наполняетесь энтузиа...,2018-01-06,12
500,Гороскоп на 19 июня 2016 года,https://dommagii.com/horoscop/2016-06-19,"[ОВЕН (21.03 - 20.04)\nВаш потенциал, особенно...",2016-06-19,12
1000,Гороскоп на 13 марта 2014,https://dommagii.com/horoscop/2014-03-13,[ОВЕН (21.03 - 20.04)\nОчень сложно сегодня уд...,2014-03-13,12
1500,Гороскоп на 31 июля 2013,https://dommagii.com/horoscop/2013-07-31,[ОВЕН (21.03 - 20.04)\nЗвезды решили проверить...,2013-07-31,12
2000,Гороскоп на 26 декабря 2012,https://dommagii.com/horoscop/2012-12-26,[ОВЕН (21.03 - 20.04)\nОвны в этот день получа...,2012-12-26,12


In [106]:
df_three[0:1].articles.values[0]

['ОВЕН (21.03 - 20.04)\nВы наполняетесь энтузиазмом к новой инициативе, проекту или отношениям, дорогой Овен, но вы в какой-то мере отнеслись к обязанностям сегодня, чтобы присутствовать, прежде чем вы сможете начать. Получение хлопот и обязанностей с вашего пути делает больше всего смысла, чтобы вы могли погрузиться в проект или отношения без вины позже. Это потрясающая неделя для общения с другими, в том числе с интимным партнером, а также с консультантами и деловыми контактами, которые могут указать вам в правильном направлении или поощрять или вдохновлять новое предприятие. Интуиция отличная для деловых и финансовых вопросов, но сегодня, позаботьтесь о практической стороне вещей.',
 'ТЕЛЕЦ (21.04 - 20.05)\nЛуна сейчас перемещается в ваш сектор игры, романтики и творчества, дорогой Телtц, и вы настроены немного повеселиться. Тем не менее, соединение Cолнца и Сатурна действующее сегодня, может напоминать вам о ваших обязанностях, и, возможно, вы любезны, так как это помогает вам подд

In [184]:
def get_separate_sign(article, i):
    
    article = [el.strip() for el in article]
    
    return ' '.join(article[i:i+1]).split('\n')[1:][0:]
            
for i, sign in enumerate(['Овен','Телец','Близнецы','Рак','Лев','Дева','Весы','Скорпион','Стрелец','Козерог','Водолей','Рыбы']):
    df_three[sign] = df_three['articles'].apply(get_separate_sign, i=i)
for sign in ['Овен','Телец','Близнецы','Рак','Лев','Дева','Весы','Скорпион','Стрелец','Козерог','Водолей','Рыбы']:
    df_three[sign] = df_three[sign].apply(lambda x: str(x[0:][0:])[2:-2])

In [185]:
df_three['Рыбы'][0:1].values

array(['Эта неделя отлично подходит для обучения и руководства другими людьми и для обмена идеями или убеждениями, дорогие Рыбы. Фактически, вы можете попытаться построить мосты таким образом. Вы, вероятно, найдете творческие способы объединить более скучные задачи с более предприимчивыми или энергичными занятиями. Ваше понимание того, какие проекты и ходы имеют истинный потенциал, сейчас имеет смысл. Вы можете быть очень взволнованы новой интересующей темой, изучать что-то новое. Тем не менее, обязанности требуют вашего внимания сейчас, но вы наверняка почувствуете себя намного лучше, если будете выполнять свои обязанности, чем если бы вы их отложили. Используйте сегодня, чтобы позаботиться о практической стороне вещей.'],
      dtype=object)

In [192]:
df_three[::500]

,titles,links,articles,date,len_articles,Овен,Телец,Близнецы,Рак,Лев,Дева,Весы,Скорпион,Стрелец,Козерог,Водолей,Рыбы
0,Гороскоп на 6 января 2018 год,https://dommagii.com/horoscop/2018-01-06,[ОВЕН (21.03 - 20.04)\nВы наполняетесь энтузиа...,2018-01-06,12,Вы наполняетесь энтузиазмом к новой инициативе...,"Луна сейчас перемещается в ваш сектор игры, ро...",Следующие несколько дней отлично подходят для ...,Отличная энергия с вами для подключения и обме...,Сегодня Луна выходит из вашего знака и попадае...,Следующие несколько дней принесут отличную эне...,"Сегодня Луна переходит в ваш секретный сектор,...",Ближайшие несколько дней являются мощными для ...,Вы недавно наслаждались Новолунием в своем зна...,Эта неделя может быть отличной для подключения...,"Эта неделя сильна для формулирования целей, св...",Эта неделя отлично подходит для обучения и рук...
500,Гороскоп на 19 июня 2016 года,https://dommagii.com/horoscop/2016-06-19,"[ОВЕН (21.03 - 20.04)\nВаш потенциал, особенно...",2016-06-19,12,"Ваш потенциал, особенно жизненный, сейчас нахо...",Будьте готовы к разным рода неожиданностям: вс...,Негативные тенденции прошлых дней продолжатся:...,Отдохните от суеты и спешки. Проведите день сп...,Немного отдохните от работы и проведите романт...,Без боязни проводите операции с ценными бумага...,Ваше здоровье сейчас не в самом лучшем состоян...,"Проведите сегодня день активно, весело и творч...",Сегодня не самый лучший день для реализации но...,"Решите, наконец, для себя многие важные и не т...",Работа занимает сейчас слишком много времени и...,Вы будете выглядеть оптимистичным и дружелюбны...
1000,Гороскоп на 13 марта 2014,https://dommagii.com/horoscop/2014-03-13,[ОВЕН (21.03 - 20.04)\nОчень сложно сегодня уд...,2014-03-13,12,"Очень сложно сегодня удержаться от агрессии, н...",Ваша энергия сегодня возросла до предела. Прил...,Обороняйте свои принципы и взгляды на жизнь. В...,Если давно планировали путешествие или команди...,Тревожность и волнения сегодня будут вас сопро...,Ждите крупных покупок и расходов. Стоит заране...,"Можно сказать, что день довольно таки благопри...",Положительные тенденции прошлой недели продолж...,Вселенная настоятельно советует отправиться в ...,Не стоит всегда анализировать каждый шаг и пос...,"День как будто создан для экспериментов, новых...","Будет расти напряжение, проблемы станут появля..."
1500,Гороскоп на 31 июля 2013,https://dommagii.com/horoscop/2013-07-31,[ОВЕН (21.03 - 20.04)\nЗвезды решили проверить...,2013-07-31,12,Звезды решили проверить вас на прочность и пос...,Подходите к каждому заданию с ответственностью...,"К каждому новому человеку, который хочет попас...",Если вы долго вынашивали интересную идею или р...,Звезды настроены к вам неблагоприятно. Постара...,Не стоит начинать сегодня новое мероприятия ил...,День обещает успех и признание любому вашему н...,"Скорее всего, день будет неярким и даже где-то...",На работе придерживайтесь тактики «Тише едешь ...,Если ваша работа связана с финансовой стороной...,Жизненный потенциал находится на приличном уро...,Сегодня будет происходить множество двойственн...
2000,Гороскоп на 26 декабря 2012,https://dommagii.com/horoscop/2012-12-26,[ОВЕН (21.03 - 20.04)\nОвны в этот день получа...,2012-12-26,12,Овны в этот день получат отличный шанс поднять...,"Достаточно тяжелый день, чреватый снижением жи...","К сожалению, сегодня все планы и идеи Близнецо...","Ракам сегодня, возможно, придется пересмотреть...","Будьте готовы к тому, что вероятно ухудшение о...","Гармоничный день, наполненный хорошими отношен...",Весам сегодня желательно тщательно проанализир...,Скорпионы в это день смогут привлечь внимание ...,Сегодня Стрельцы могут смело приниматься за но...,Этот день потребует от Козерогов осторожности....,Отличный день для общения с сотрудниками и нач...,Рыбы сегодня смогут добиться успеха в решении ...


In [191]:
df_three = df_three.dropna()

In [194]:
df_three.to_csv('df_three_dommagii.csv', index=False, sep=";")